Train grid based and pointwise vol surface vae

In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
sys.path.insert(0, os.getenv('SRC_PATH'))

import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from src.volsurface import GridInterpVolSurface, KernelVolSurface

import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from src.train import Trainer

import json

In [2]:
CSV_PATH = os.getenv('CSV_PATH')
SRC_PATH = os.getenv('SRC_PATH')
os.chdir(SRC_PATH)

In [3]:
from src.utils.logger import setup_logger

# Set up logger
logger = setup_logger(__name__, level="INFO")

## grid


In [4]:
model_name = "vae_v2"
train_model = True
load_model = False
save_model = True
data_dir = CSV_PATH + "/predicted_vol_surfaces.json"  # Path to the volatility surfaces dataset
batch_size = 32
epochs = 10

# Initialize the trainer
trainer = Trainer(model_name)

if train_model:
    trainer.create_model()

    if load_model:
        trainer.load_model(f"params/{trainer.model_name}.pth")
    else:
        # Create dataset
        transform = transforms.Compose([transforms.ToTensor()])
        # Load the JSON file
        with open(data_dir, "r") as f:
            data = json.load(f)

        vol_surfaces = []
        for key in data:
            surface = torch.tensor(data[key], dtype=torch.float32)
            vol_surfaces.append(surface.flatten())  # Flatten 2D to 1D

        data_tensor = torch.stack(vol_surfaces)
        dummy_labels = torch.zeros(len(data_tensor))

        dataset = TensorDataset(data_tensor, dummy_labels)
        train_loader = DataLoader(
            dataset, 
            batch_size=trainer.batch_size,
            shuffle=True
        )

        # Train the model
        for epoch in range(epochs):
            logger.info(f"Epoch {epoch + 1}/{epochs}")
            trainer.train(train_loader)

        if save_model:
            torch.save(
                trainer.model.state_dict(), f"params/{trainer.model_name}.pth"
            )

    # Evaluate the model
    trainer.evaluate("output")

[2025-04-15 23:07:12] [INFO] src.train: Using device: mps


[2025-04-15 23:07:12] [INFO] __main__: Epoch 1/10


torch.Size([128, 72])


[2025-04-15 23:07:12] [INFO] src.train: Loss: 7.7757
[2025-04-15 23:07:12] [INFO] __main__: Epoch 2/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 7.2347
[2025-04-15 23:07:12] [INFO] __main__: Epoch 3/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 6.6939
[2025-04-15 23:07:12] [INFO] __main__: Epoch 4/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 6.2236
[2025-04-15 23:07:12] [INFO] __main__: Epoch 5/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 5.7365
[2025-04-15 23:07:12] [INFO] __main__: Epoch 6/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 5.2882
[2025-04-15 23:07:12] [INFO] __main__: Epoch 7/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 4.8233
[2025-04-15 23:07:12] [INFO] __main__: Epoch 8/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 4.5008
[2025-04-15 23:07:12] [INFO] __main__: Epoch 9/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 4.0467
[2025-04-15 23:07:12] [INFO] __main__: Epoch 10/10
[2025-04-15 23:07:12] [INFO] src.train: Loss: 3.6976


torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])
torch.Size([128, 72])
torch.Size([124, 72])


## pointwise

In [15]:
df_pw = pd.read_csv(CSV_PATH + "/predicted_vol_surfaces_pw.csv")
df_pw

,delta,maturity,2022-08-31 00:00:00,2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,2022-09-09 00:00:00,2022-09-12 00:00:00,...,2023-08-18 00:00:00,2023-08-21 00:00:00,2023-08-22 00:00:00,2023-08-23 00:00:00,2023-08-24 00:00:00,2023-08-25 00:00:00,2023-08-28 00:00:00,2023-08-29 00:00:00,2023-08-30 00:00:00,2023-08-31 00:00:00
0,0.1,1,0.183529,0.199003,0.201057,0.239185,0.235878,0.248962,0.180068,0.287897,...,0.150547,0.125697,0.126810,0.183370,0.126245,0.134464,0.112328,0.144099,0.109429,0.097757
1,0.2,1,0.191890,0.197325,0.200662,0.229182,0.220532,0.221847,0.184270,0.259231,...,0.142648,0.122651,0.123355,0.163373,0.121746,0.128554,0.110913,0.133533,0.107407,0.097726
2,0.3,1,0.192045,0.189589,0.194189,0.205863,0.195425,0.190152,0.182083,0.197920,...,0.124447,0.116373,0.116634,0.122746,0.114962,0.115809,0.108748,0.112394,0.104063,0.099404
3,0.4,1,0.181409,0.181387,0.185697,0.192886,0.184825,0.181130,0.176914,0.181992,...,0.117958,0.114931,0.115039,0.112504,0.114980,0.111931,0.108991,0.107149,0.104770,0.102523
4,0.5,1,0.179949,0.180021,0.184481,0.189005,0.182152,0.179177,0.175035,0.178026,...,0.117113,0.115201,0.115175,0.114036,0.115874,0.112991,0.110764,0.109173,0.107558,0.105901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.5,720,0.326925,0.322228,0.327065,0.337686,0.315483,0.305736,0.299342,0.305816,...,0.246033,0.240683,0.238543,0.226742,0.238890,0.222914,0.214898,0.204893,0.196526,0.193651
68,0.6,720,0.335651,0.330776,0.334859,0.345558,0.327007,0.321295,0.314500,0.316012,...,0.258559,0.252490,0.251579,0.241190,0.252260,0.238032,0.230511,0.220779,0.213506,0.210191
69,0.7,720,0.353070,0.347622,0.351380,0.357807,0.343726,0.339076,0.334220,0.331717,...,0.271908,0.267161,0.268047,0.260454,0.266524,0.257150,0.250803,0.243068,0.238549,0.235913
70,0.8,720,0.350371,0.350921,0.346709,0.325683,0.327348,0.325610,0.326479,0.325305,...,0.324609,0.319901,0.322514,0.322831,0.322696,0.318030,0.312629,0.316254,0.316223,0.318413


In [4]:
model_name = "vae_v3"
train_model = True
load_model = False
save_model = True
data_dir = CSV_PATH + "/predicted_vol_surfaces_pw.csv"  # Path to the volatility surfaces point wise dataset
batch_size = 32
epochs = 10

# Initialize the trainer
trainer = Trainer(model_name)

if train_model:
    trainer.create_model()

    if load_model:
        trainer.load_model(f"params/{trainer.model_name}.pth")
    else:
        # Create dataset
        transform = transforms.Compose([transforms.ToTensor()])
        df_pw = pd.read_csv(CSV_PATH + "/predicted_vol_surfaces_pw.csv")
        vol_surfaces = []
        columns = [col for col in df_pw.columns if col not in ['delta', 'maturity']]
        for col in columns:
            for (k, t) in df_pw[['delta', 'maturity']].values:
                surface = df_pw[col].values
                surface = np.append(surface, [k, t])
                surface = torch.tensor(surface, dtype=torch.float32)
                vol_surfaces.append(surface)

        # Stack the list of tensors into a single tensor
        data_tensor = torch.stack(vol_surfaces)
        dummy_labels = torch.zeros(len(data_tensor))

        # Create the dataset and DataLoader
        dataset = TensorDataset(data_tensor, dummy_labels)
        train_loader = DataLoader(
            dataset,
            batch_size=trainer.batch_size,
            shuffle=True
        )

        # Train the model
        for epoch in range(epochs):
            logger.info(f"Epoch {epoch + 1}/{epochs}")
            trainer.train(train_loader)

        if save_model:
            torch.save(
                trainer.model.state_dict(), f"params/{trainer.model_name}.pth"
            )

    # Evaluate the model
    trainer.evaluate("output")

[2025-04-16 15:26:03] [INFO] src.train: Using device: mps


[2025-04-16 15:26:03] [INFO] __main__: Epoch 1/10
/Users/zlx/Desktop/2025Spring/COS513/VAE-VolSurface/src/models/vae_pw.py:61: UserWarning: Using a target size (torch.Size([128, 74])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  MSE = nn.functional.mse_loss(
/Users/zlx/Desktop/2025Spring/COS513/VAE-VolSurface/src/models/vae_pw.py:61: UserWarning: Using a target size (torch.Size([96, 74])) that is different to the input size (torch.Size([96, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  MSE = nn.functional.mse_loss(
[2025-04-16 15:26:04] [INFO] src.train: Loss: 86307.3384
[2025-04-16 15:26:04] [INFO] __main__: Epoch 2/10
[2025-04-16 15:26:04] [INFO] src.train: Loss: 86306.8983
[2025-04-16 15:26:04] [INFO] __main__: Epoch 3/10
[2025-04-16 15:26:05] [INFO] src.train: Loss: 86306.8700
[2025-04-16 15:26:05] [

In [12]:
tmp = df_pw[col].values
np.append(tmp, [k, t])

array([9.77565557e-02, 9.77263326e-02, 9.94039128e-02, 1.02523128e-01,
       1.05900803e-01, 1.09591129e-01, 1.30912203e-01, 1.25717487e-01,
       9.81518359e-02, 9.81278260e-02, 1.00326671e-01, 1.04201310e-01,
       1.07691153e-01, 1.13143632e-01, 1.23594452e-01, 1.34983357e-01,
       9.91077674e-02, 9.94593216e-02, 1.02947568e-01, 1.08044352e-01,
       1.12098768e-01, 1.18957566e-01, 1.28789765e-01, 1.43792538e-01,
       1.00008662e-01, 1.00957957e-01, 1.06244338e-01, 1.12753927e-01,
       1.17518213e-01, 1.26165597e-01, 1.38313185e-01, 1.52732907e-01,
       1.01566149e-01, 1.03128564e-01, 1.10441852e-01, 1.18735493e-01,
       1.24649869e-01, 1.35326963e-01, 1.49763451e-01, 1.63624060e-01,
       1.03596899e-01, 1.05962438e-01, 1.15909020e-01, 1.26356115e-01,
       1.33556410e-01, 1.46622772e-01, 1.63799323e-01, 1.82479528e-01,
       1.06588170e-01, 1.09943708e-01, 1.23365068e-01, 1.36826074e-01,
       1.45793184e-01, 1.61581529e-01, 1.82076331e-01, 1.98924029e-01,
      

In [10]:
tmp

72